<a href="https://colab.research.google.com/github/babo-dev/rnn-projects/blob/main/rnn_mnist_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_classes = 10
num_epochs = 3
batch_size = 128
learning_rate = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 4

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.04MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.9MB/s]


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # x: (n, 28, 28), h0: (2, n, 128)
        out, _ = self.rnn(x, h0)

        out = out[:, -1, :]
        # out: (n, 128)

        out = self.fc(out)
        # out: (n, 10)
        return out

In [ ]:
class CustomOneLayerRnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomOneLayerRnn, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        h = torch.zeros(x.size(0), self.hidden_size).to(device)
        for i in range(x.size(1)):
            combined = torch.cat((x[:, i, :], h), dim=1)
            h = torch.tanh(self.i2h(combined))
        out = self.i2o(torch.cat((x[:, -1, :], h), dim=1))
        out = self.softmax(out)
        return out

In [ ]:
class CustomMultiLayerRnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CustomMultiLayerRnn, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.i2h = nn.ModuleList(
            [nn.Linear(input_size + hidden_size, hidden_size) if i == 0 else nn.Linear(hidden_size+hidden_size, hidden_size) for i in range(num_layers)]
            )

        self.i2o = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, images):
        h = [torch.zeros(images.size(0), self.hidden_size).to(device) for _ in range(self.num_layers)]

        for i in range(images.size(1)):
            for layer in range(self.num_layers):
                if layer == 0:
                    combined = torch.cat((images[:, i, :], h[layer]), dim=1)
                else:
                    combined = torch.cat((h[layer - 1], h[layer]), dim=1)
                h[layer] = torch.tanh(self.i2h[layer](combined))

        output = self.i2o(h[-1])
        output = self.softmax(output)
        return output

In [ ]:
model = CustomMultiLayerRnn(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
model = CustomOneLayerRnn(input_size, hidden_size, num_classes).to(device)

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
n_total_steps

469

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/3], Step [100/469], Loss: 1.0209
Epoch [1/3], Step [200/469], Loss: 0.8072
Epoch [1/3], Step [300/469], Loss: 0.5733
Epoch [1/3], Step [400/469], Loss: 0.5730
Epoch [2/3], Step [100/469], Loss: 0.2393
Epoch [2/3], Step [200/469], Loss: 0.3862
Epoch [2/3], Step [300/469], Loss: 0.3359
Epoch [2/3], Step [400/469], Loss: 0.2526
Epoch [3/3], Step [100/469], Loss: 0.2609
Epoch [3/3], Step [200/469], Loss: 0.2693
Epoch [3/3], Step [300/469], Loss: 0.2228
Epoch [3/3], Step [400/469], Loss: 0.1381


In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 94.25 %
